# Pandas - Python Data Analysis

Pandas é uma biblioteca para trabalhar com a parte de preparação de dados. De certa forma, suas operações são similares às operações SQL em bancos de dados, operando sobre dados tabulares, com a principal diferença sendo que pandas é utilizado em memória e localmente, enquanto bases SQL geralmente são operadas em disco e em servidores.

Os dados em Pandas são organizados em `Series`, que seria equivalente a uma coluna de uma tabela, e `DataFrames`, que representam as tabelas propriamente ditas. De certa forma, os dados em uma `Series` são como um `ndarray` do Numpy.

Um `DataFrame` pode ser criado de várias formas, mas comumente é feito a partir de dados em um banco de dados SQL, um arquivo Excel, de uma matriz ou vetores Numpy, ou um arquivo CSV.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Shapes

Antes de entrar em pandas propriamente dito, uma nota sobre _shapes_ em Numpy.

In [ ]:
x = np.array([[5, 2],
              [1, 3],
              [7, 8]])
x.shape

In [ ]:
x.T

In [ ]:
x.T.shape

In [ ]:
x2 = np.array([5, 2, 1, 3, 7, 8]).reshape(3, 2)
x2

In [ ]:
np.array_equal(x, x2)

Pergunta: por que `np.array_equal(x, x2)` e não `x == x2`?

In [ ]:
x3 = x.reshape(-1, 1)
print(x3.shape)
x3

In [ ]:
x4 = x.reshape(1, -1)
print(x4.shape)
x4

In [ ]:
x5 = x.ravel() # igual .reshape(-1)
print(x5.shape)
x5

In [ ]:
print(np.array_equal(x3, x4))
print(np.array_equal(x3, x5))
print(np.array_equal(x4, x5))
print(np.array_equal(x5, x.reshape(-1)))

- O tamanho de cada dimensão é guardado na variável `shape` do vetor
- Um vetor-linha (N, 1) é diferente de um vetor-coluna (1, N)
  - Geralmente usamos vetor-linha para representar uma amostra, e vetor-coluna para representar um atributo
- E ambos são diferentes de um vetor unidimensional, que é ambíguo se representa uma amostra ou atributo
- Quando damos reshape, o valor -1 indica "o valor necessário para ser compatível"; somente uma dimensão pode ser reshaped pra -1, caso contrário o reshape é ambíguo
- A função `ravel` deixa a dimensão de um vetor com (N,), e é só um atalho para `reshape(-1)`.

In [ ]:
x1 = np.array([5,    3,    8,     0,    4])
x2 = np.array([0.5,  3.7,  1.6,   2,    7.8])
y = np.array( ['Sim', 'Sim', 'Não', 'Sim', 'Não'])

def mapeia_cor(val):
  if val == 'Sim':
    return 'red'
  else:
    return 'black'

cor = np.vectorize(mapeia_cor)(y)
print('Cores:', cor, 'Tipo:', cor.dtype)
plt.scatter(x1, x2, c=cor, s=100);

In [ ]:
dados = pd.DataFrame({'Quantidade de crianças': x1,
                      'Renda familiar (salários mínimos)': x2,
                      'Possui débito': pd.Categorical(y)})
dados

In [ ]:
dados.shape

In [ ]:
len(dados)

In [ ]:
dados.dtypes

In [ ]:
dados.head(2)

In [ ]:
dados.describe()

In [ ]:
dados.describe(include=['category'])

## Indexação/seleção

Uma vantagem do pandas são as formas de indexação. Antes de entrar nos detalhes do pandas, vamos ver um novo tipo de indexação para vetores do numpy que ainda não discutimos: coleção de índices.

In [ ]:
x1

In [ ]:
indices = np.array([2, 4])
x1[indices]

In [ ]:
# também funciona com listas como índice, e a sintaxe abaixo é comum
x1[[2, 4]]

Assim, vimos as seguintes formas de indexação:

- Índices direto: `x1[2] == 8`
- _Slicing_/intervalos: `x1[2:4] == [8, 0]`
- Máscaras booleanas: `x1[[True, False, True, False, False]] == [5, 8]`
- Coleção de índices: `x1[[2, 4]] == [8, 4]`

Todas essas formas também funcionam com pandas. A diferença é que em pandas, os índices **não necessariamente são inteiros começando em zero**.

In [ ]:
dados.columns

In [ ]:
dados.index

No caso das linhas (`dados.index`), o índice é o intervalo entre 0 e 4, como indicado pelo `range(0, 5)`. Mas no caso das colunas, o **índice é o próprio nome das colunas**. Essa é a forma mais comum de tratar de DataFrames, com as colunas sendo indexadas por texto e as linhas sendo indexadas por inteiros.

Além disso, apesar do `DataFrame` ser uma matriz, quando um único índice é passado, a dimensão pode ser qualquer uma das duas dependendo de qual índice se encaixa.

In [ ]:
dados['Quantidade de crianças']

In [ ]:
dados[:2]

Quando queremos selecionar nos dois eixos simultaneamente, é necessário colocar a propriedade `.loc` na frente do DataFrame para desligar o modo "smart" da indexação. Nesse caso, o primeiro elemento sempre é as linhas e o segundo as colunas, assim como em Numpy. A diferença é que o elemento após o `:` no _slice_ é incluso também (e não até um antes).

Ainda podemos colocar `.iloc` para tratar os índices como se fossem inteiros. Isso não altera a forma de indexação, essa transformação é somente para o comando que utiliza o `iloc` (veja abaixo que o nome da Series ainda é "Possui débito").

Lembre que é possível criar um _slice_ sem nem início nem fim (representado somente por `:`); nesse caso, todos os elementos daquele eixo são selecionados.

In [ ]:
dados.loc[:2,
          ['Quantidade de crianças',
           'Renda familiar (salários mínimos)']]

In [ ]:
dados.iloc[:, 0]

Note que é possível alterar o método de indexação, mesmo para as linhas.

In [ ]:
dados.index = pd.Index(['Alfredo', 'Beatriz', 'Carlos', 'Diana', 'Eduardo'])
dados

In [ ]:
dados.loc['Beatriz', 'Quantidade de crianças']

In [ ]:
# no caso que ambos os índices são de texto
# o modo smart assume que trata de colunas
dados['Possui débito']

In [ ]:
# só voltando para configuração padrão
dados.index = pd.RangeIndex(0, 5)

Só para reforçar: todas as formas de indexação do numpy funcionam com pandas. E como essas operações retornam um novo DataFrame$^1$ e não alteram o original, é possível "encadear" as operações. Dessa forma, basicamente temos as operações SELECT e WHERE do SQL (seleção vertical, seleção horizontal e filtragem por condição), além de algumas operações de agregação que o Numpy oferece (média, soma, etc).

$^1$: mais ou menos. Geralmente se retorna uma _view_ do DataFrame original, mas pode depender da operação e mesmo da versão do pandas.

In [ ]:
dados[dados['Possui débito'] == 'Sim']

In [ ]:
(dados[dados['Quantidade de crianças'] >= 4]
 ['Renda familiar (salários mínimos)'])

In [ ]:
dados.loc[dados['Quantidade de crianças'] >= 4, 'Renda familiar (salários mínimos)']

Para utilizar os operadores lógicos **e** e **ou** dentro de uma indexação, são utilizados os símbolos `&` e `|`, respectivamente. É obrigatório colocar parêntesis em cada um dos lados, no entanto.

In [ ]:
dados[ (dados['Possui débito'] == 'Não') &
    (dados['Quantidade de crianças'] <= 4)]

## Atribuição

É possível atribuir valores para células diretamente, assim como com vetores. Mais interessante, se a atribuição é num índice textual que ainda não existe, uma nova coluna é criada.

In [ ]:
dados

In [ ]:
dados.loc[3, 'Possui débito'] = 'Não'
dados

In [ ]:
dados['Tem carro'] = pd.Categorical(['Não', 'Sim', 'Sim', 'Não', 'Sim'])
dados

## Visualizações e estatísticas básicas

O pandas já nos dá algumas funções prontas para visualização e estatísticas que são comumente aplicadas.

In [ ]:
dados.hist(bins=3);

In [ ]:
dados.plot();

In [ ]:
dados.plot(kind='bar');
#dados.plot.bar(); # equivalente

In [ ]:
dados.plot(kind='box');

In [ ]:
# o parâmetro s é o tamanho do marcador
# diferente do padrão matplotlib, o pandas cria um gráfico por chamada de função, e não um por célula
# para criar múltiplos gráficos juntos, é utilizado o parâmetro ax (entenda-se "utilize o mesmo eixo")

ax = (dados[dados['Possui débito'] == 'Sim'].plot
      .scatter(x='Quantidade de crianças',
               y='Renda familiar (salários mínimos)',
               c='red', marker='*', s=80,
               label='Com débito'))
dados[dados['Possui débito'] == 'Não'].plot.scatter(x='Quantidade de crianças', y='Renda familiar (salários mínimos)', c='black', marker='*', s=80, label='Sem débito', ax=ax);

In [ ]:
dados.corr()

In [ ]:
dados.cov()

In [ ]:
dados.kurtosis()

E por fim se lembre, cada coluna é uma `Series`, e as series funcionam como `ndarrays`. Logo, operações vetoriais funcionam com as colunas de um `DataFrame`.

In [ ]:
(dados['Quantidade de crianças']
 / dados['Renda familiar (salários mínimos)'])